# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# your code here

Sub Problem 1: aggregate the  amount_spent for unique customers

In [12]:
aggcust = orders.groupby("CustomerID").agg({"amount_spent":"sum"})
aggcust

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [80]:
import numpy as np
agg_expense_95 = np.percentile(aggcust["amount_spent"], 95)
agg_expense_95
agg_expense_75 = np.percentile(aggcust["amount_spent"], 75)
agg_expense_75
agg_expense_max = aggcust["amount_spent"].max()
agg_expense_min = aggcust["amount_spent"].min()



labels = ["Others","Preferred","VIP"]
bins = pd.cut(aggcust["amount_spent"],[agg_expense_min,agg_expense_75,agg_expense_95,agg_expense_max],labels = labels)
bins.value_counts()

Others       3253
Preferred     868
VIP           217
Name: amount_spent, dtype: int64

In [81]:
aggcust["bins_d"]=bins
aggcust

,amount_spent,bins_d
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Others
...,...,...
18280,180.60,Others
18281,80.82,Others
18282,178.05,Others


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [82]:
cust_country=orders[["CustomerID","Country"]].drop_duplicates()
cust_country

,CustomerID,Country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,France
46,13748,United Kingdom
65,15100,United Kingdom
...,...,...
394965,13436,United Kingdom
395041,15520,United Kingdom
395777,13298,United Kingdom
396506,14569,United Kingdom


In [129]:
aggcust_countries = pd.merge(left=aggcust,
                               right = cust_country,
                               how = "inner",
                               left_on = "CustomerID",
                               right_on = "CustomerID")
aggcust_countries

,CustomerID,amount_spent,bins_d,Country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,Preferred,Iceland
2,12348,1797.24,Preferred,Finland
3,12349,1757.55,Preferred,Italy
4,12350,334.40,Others,Norway
...,...,...,...,...
4342,18280,180.60,Others,United Kingdom
4343,18281,80.82,Others,United Kingdom
4344,18282,178.05,Others,United Kingdom
4345,18283,2094.88,Preferred,United Kingdom


In [128]:
aggcust_countries_group= aggcust_countries[aggcust_countries["bins_d"]=="VIP"].groupby(["Country"]).agg({"bins_d":"count"})
aggcust_countries_group

,bins_d
Country,
Australia,2
Belgium,1
Channel Islands,1
Cyprus,1
Denmark,1
EIRE,2
Finland,1
France,9
Germany,10


In [126]:
aggcust_countries_group.nlargest(n=1,columns = ["bins_d"])

,bins_d
Country,
United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [131]:
# your code here
aggcust_countries_group= aggcust_countries[aggcust_countries["bins_d"]!= "Others"].groupby("Country").agg({"bins_d":"count"})
aggcust_countries_group

,bins_d
Country,
Australia,4
Austria,3
Belgium,12
Canada,1
Channel Islands,4
Cyprus,4
Denmark,3
EIRE,3
Finland,5


In [117]:
aggcust_countries_group.nlargest(n=1,columns = ["bins_d"])

,bins_d
Country,
United Kingdom,932
